# 問題分析と実装

## 1. 問題分析結果

### 競技プログラミング視点
- **制約分析**: n ≤ 10^5 → O(√n) の約数列挙で十分
- **最速手法**: √n までループして約数を収集、桁和を組み込み関数で計算
- **メモリ最小化**: 約数の数は高々O(√n)、桁和計算は即座に実行
- **CPython最適化**: `sum()`と文字列変換、`max()`のkey引数を活用

### 業務開発視点
- **型安全設計**: 入力検証、厳密な型ヒント
- **エラーハンドリング**: 不正入力の検証
- **可読性**: 明確な関数分割、詳細なdocstring

### Python特有分析
- **データ構造選択**: リストで約数を保持（順序不要、重複なし）
- **標準ライブラリ活用度**: 組み込み関数`sum()`, `max()`を効果的に使用
- **CPython最適化度**: 文字列変換とジェネレータ式でC実装を活用

## 2. 採用アルゴリズムと根拠

### アルゴリズム比較表

|アプローチ|時間計算量|空間計算量|Python実装コスト|可読性|標準ライブラリ活用|CPython最適化|備考|
|---------|---------|---------|---------------|------|----------------|------------|-----|
|√n約数列挙|O(√n)|O(d)|低|★★★|sum(), max()|適|dは約数の個数|
|全探索|O(n)|O(d)|低|★★☆|同上|適|非効率|

**選択理由**: O(√n)で全約数を列挙可能。桁和計算は定数時間（桁数は高々log n）。Pythonの組み込み関数を最大活用。

**Python最適化戦略**:
- `sum(int(d) for d in str(num))`: ジェネレータ式とC実装のsumを活用
- `max(divisors, key=...)`: C実装のmax関数でソート不要

## 3. 実装パターン

```python
from typing import List, Tuple

class Solution:
    """
    最良の約数を見つける問題の解決クラス
    
    競技プログラミング向けと業務開発向けの2パターンを提供
    """
    
    def findBestDivisor(self, n: int) -> int:
        """
        業務開発向け実装（型安全・エラーハンドリング重視）
        
        Args:
            n: 正の整数（1 ≤ n ≤ 10^5）
            
        Returns:
            最良の約数（桁和が最大、同じなら最小値）
            
        Raises:
            ValueError: 入力値が制約を満たさない場合
            TypeError: 入力型が不正な場合
            
        Time Complexity: O(√n + d)
        Space Complexity: O(d)
        """
        # 1. 入力検証
        self._validate_input(n)
        
        # 2. 約数を列挙
        divisors = self._find_divisors(n)
        
        # 3. 最良の約数を選択
        best_divisor = self._select_best_divisor(divisors)
        
        return best_divisor
    
    def findBestDivisorCompetitive(self, n: int) -> int:
        """
        競技プログラミング向け最適化実装
        
        Time Complexity: O(√n)
        Space Complexity: O(d)
        """
        divisors: List[int] = []
        i = 1
        while i * i <= n:
            if n % i == 0:
                divisors.append(i)
                if i != n // i:
                    divisors.append(n // i)
            i += 1
        
        # 桁和が最大、同値なら最小値を選択
        return max(divisors, key=lambda x: (sum(int(d) for d in str(x)), -x))
    
    def _validate_input(self, n: int) -> None:
        """型安全な入力検証"""
        if not isinstance(n, int):
            raise TypeError(f"Input must be integer, got {type(n)}")
        
        if n < 1 or n > 10**5:
            raise ValueError(f"Input must be in range [1, 10^5], got {n}")
    
    def _find_divisors(self, n: int) -> List[int]:
        """
        nの全約数を列挙
        
        Args:
            n: 正の整数
            
        Returns:
            約数のリスト
        """
        divisors: List[int] = []
        i = 1
        # √nまでループして約数をペアで収集
        while i * i <= n:
            if n % i == 0:
                divisors.append(i)
                # i ≠ n/i の場合のみ追加（平方数対策）
                if i != n // i:
                    divisors.append(n // i)
            i += 1
        
        return divisors
    
    def _digit_sum(self, num: int) -> int:
        """
        数値の桁和を計算
        
        Args:
            num: 正の整数
            
        Returns:
            各桁の合計
        """
        # 文字列変換とsum()でCPython最適化
        return sum(int(digit) for digit in str(num))
    
    def _select_best_divisor(self, divisors: List[int]) -> int:
        """
        最良の約数を選択
        
        Args:
            divisors: 約数のリスト
            
        Returns:
            最良の約数（桁和最大、同値なら最小値）
        """
        # max()のkey引数を使用
        # タプルで優先順位: (桁和が大きい, 値が小さい)
        return max(divisors, key=lambda x: (self._digit_sum(x), -x))


# メイン処理
if __name__ == '__main__':
    n = int(input().strip())
    solution = Solution()
    # 競技プログラミング版を使用（高速）
    result = solution.findBestDivisorCompetitive(n)
    print(result)
```

## 4. 検証

### 境界値テスト
- **n = 1**: 約数は1のみ → 出力: 1
- **n = 12**: 約数 {1,2,3,4,6,12}、桁和 {1,2,3,4,6,3} → 最大6 → 出力: 6 ✓
- **n = 100**: 約数に10(桁和1), 20(桁和2), 25(桁和7), 50(桁和5)等 → 桁和最大を選択

### Python特有の最適化ポイント
1. **組み込み関数**: `sum()`, `max()` はC実装で高速
2. **ジェネレータ式**: `sum(int(d) for d in str(x))` はメモリ効率的
3. **文字列変換**: 桁和計算で算術演算より簡潔かつ高速
4. **タプル比較**: `max(key=lambda x: (a, -b))` で複数条件ソート

# コード全体の目的

**入力:** 整数 `n`
**出力:**

* 約数の中で **桁和（各桁の合計）が最大のもの**
* 桁和が同じなら **数値が小さい方**

---

# コード本体

```python
def findBestDivisor(n: int) -> int:
```

`n` の「最良の約数」を求める関数です。

---

# 計算量の意図（docstring）

```python
"""
Time Complexity: O(√n)
Space Complexity: O(d) where d is number of divisors
"""
```

### ✔ 時間計算量 `O(√n)`

約数探索を √n までに制限して高速化。

### ✔ 空間計算量 `O(d)`

約数の個数 `d` 分だけメモリ使用。

---

# 約数収集ロジック（√n 最適化）

```python
divisors = []
i = 1
```

---

```python
while i * i <= n:
```

### なぜ √n までで良い？

約数は **ペア（i, n//i）** で現れるため、√n まで探索すれば十分。

---

```python
if n % i == 0:
    divisors.append(i)
```

`i` が約数なら追加。

---

```python
if i != n // i:
    divisors.append(n // i)
```

### ✔ 平方数対策

* `n = 36` のとき `i = 6` → `6 * 6`
* 同じ約数を **2回追加しない** ように防止

---

### ✔ 結果として

**全ての約数を漏れなく収集**

---

# 約数の選択ルール（最重要）

```python
return max(divisors, key=lambda x: (sum(int(d) for d in str(x)), -x))
```

---

## 評価基準（タプル比較）

```python
(sum_of_digits(x), -x)
```

### ① 桁和が大きいものを優先

```python
sum(int(d) for d in str(x))
```

例:

* 84 → 8+4 = 12
* 96 → 9+6 = 15（勝ち）

---

### ② 桁和が同じなら「値が小さい方」

```python
-x
```

**max() なので、値を反転することで「小さい数を優先」**

例:

* 39 (3+9=12)
* 48 (4+8=12)

同じ桁和 → **39 を選ぶ**

---

# 実行例

### 入力

```
n = 100
```

### 約数

```
1, 2, 4, 5, 10, 20, 25, 50, 100
```

### 桁和

| 約数  | 桁和 |
| --- | -- |
| 1   | 1  |
| 2   | 2  |
| 4   | 4  |
| 5   | 5  |
| 10  | 1  |
| 20  | 2  |
| 25  | 7  |
| 50  | 5  |
| 100 | 1  |

### ✔ 最大は `25`（7）

→ 出力 `25`

---

# main 部分

```python
if __name__ == '__main__':
    n = int(input().strip())
    result = findBestDivisor(n)
    print(result)
```

標準入力から整数を受け取り、結果を出力。

---

# アルゴリズムの強み

| 項目       | 評価                 |
| -------- | ------------------ |
| 高速       | √n で約数探索           |
| 無駄がない    | 約数ペア回収             |
| Pythonic | `max(key=...)` で簡潔 |
| 競プロ向き    | 大きな n にも対応         |

---

# 改善・代替案（さらに最適化）

## メモリを使わず直接最大を更新（O(1) space）

```python
def findBestDivisor(n):
    best = 1
    best_sum = 1
    
    i = 1
    while i * i <= n:
        if n % i == 0:
            for d in (i, n // i):
                s = sum(map(int, str(d)))
                if s > best_sum or (s == best_sum and d < best):
                    best = d
                    best_sum = s
        i += 1
    
    return best
```